<h1>training phrase</h1>

In [1]:
import pandas as pd

In [2]:
df_train=pd.read_csv('data/train.csv')

In [4]:
print(df_train.head())

         date  calls   weekday  reservations_2months_advance  \
0  2014-01-31   2477    Friday                          2906   
1  2014-02-01   1600  Saturday                          1360   
2  2014-02-02   1310    Sunday                          1552   
3  2014-02-03   2673    Monday                          2135   
4  2014-02-04   2460   Tuesday                          2207   

   total_reservations  summer_break  christmas_break  special_day  dayofweek  \
0               13759             0                0            0          4   
1                9498             0                0            0          5   
2                7815             0                0            0          6   
3               15401             0                0            0          0   
4               13840             0                0            0          1   

   quarter  ...  reservations_ratio  calls_per_reservation  sin_dayofyear  \
0        1  ...            0.211192               0.18001

In [3]:
df_train.head()
df_train.info()
df_train.describe()
df_train.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 608 entries, 0 to 607
Data columns (total 59 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   date                          608 non-null    object 
 1   calls                         608 non-null    int64  
 2   weekday                       608 non-null    object 
 3   reservations_2months_advance  608 non-null    int64  
 4   total_reservations            608 non-null    int64  
 5   summer_break                  608 non-null    int64  
 6   christmas_break               608 non-null    int64  
 7   special_day                   608 non-null    int64  
 8   dayofweek                     608 non-null    int64  
 9   quarter                       608 non-null    int64  
 10  month                         608 non-null    int64  
 11  year                          608 non-null    int64  
 12  calls_lag_1                   608 non-null    float64
 13  total

date                            0
calls                           0
weekday                         0
reservations_2months_advance    0
total_reservations              0
summer_break                    0
christmas_break                 0
special_day                     0
dayofweek                       0
quarter                         0
month                           0
year                            0
calls_lag_1                     0
total_reservations_lag_1        0
reservations_2months_lag_1      0
calls_lag_7                     0
total_reservations_lag_7        0
reservations_2months_lag_7      0
calls_lag_14                    0
total_reservations_lag_14       0
reservations_2months_lag_14     0
calls_lag_30                    0
total_reservations_lag_30       0
reservations_2months_lag_30     0
calls_roll_mean_7               0
calls_roll_std_7                0
calls_roll_min_7                0
calls_roll_max_7                0
reservations_roll_mean_7        0
reservations_r

<h1>start forecasting</h1>

In [5]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import warnings
warnings.filterwarnings('ignore')

c:\isuru_dd\tellcolll\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [6]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [7]:
train_df=df_train.copy()

In [8]:
#convert date to datetime
train_df['date']=pd.to_datetime(train_df['date'])

In [9]:
# Use last 20% of training data for validation (approximately 122 days)
val_size = int(len(train_df) * 0.2)
train_data = train_df.iloc[:-val_size].copy()
val_data = train_df.iloc[-val_size:].copy()

print(f"Train set: {len(train_data)} days ({train_data['date'].min()} to {train_data['date'].max()})")
print(f"Validation set: {len(val_data)} days ({val_data['date'].min()} to {val_data['date'].max()})")

Train set: 487 days (2014-01-31 00:00:00 to 2015-06-01 00:00:00)
Validation set: 121 days (2015-06-02 00:00:00 to 2015-09-30 00:00:00)


In [10]:
# Prophet requires 'ds' (date) and 'y' (target) columns
prophet_train = pd.DataFrame({
    'ds': train_data['date'],
    'y': train_data['calls']
})

In [11]:
# Add relevant regressors - selecting most important features
# Avoid lag features and rolling stats as they would need complex future calculations
regressors = [
    'total_reservations',
    'reservations_2months_advance',
    'summer_break',
    'christmas_break',
    'special_day',
    'is_weekend',
    'dayofweek',
    'month',
    'quarter'
]

In [12]:
for reg in regressors:
    prophet_train[reg] = train_data[reg].values

print(f"Prophet training dataframe shape: {prophet_train.shape}")
print(f"Regressors included: {regressors}")

Prophet training dataframe shape: (487, 11)
Regressors included: ['total_reservations', 'reservations_2months_advance', 'summer_break', 'christmas_break', 'special_day', 'is_weekend', 'dayofweek', 'month', 'quarter']
